In [ ]:
import argparse
import pathlib
import numpy as np
import cv2
import time

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision

from PIL import Image
from PIL import Image, ImageOps

from face_detection import RetinaFace

from l2cs import select_device, draw_gaze, getArch, Pipeline, render

CWD = pathlib.Path.cwd()

# 设置参数
video_path = '../test_gaze/video_12.mp4'  # 修改为你的视频路径
output_path = '../output_gaze/video_12.mp4'
device = 'mps'  # 改为 'gpu:0' 如果有 GPU
arch = 'ResNet121'

cudnn.enabled = True

gaze_pipeline = Pipeline(
    weights=CWD / 'models' / 'L2CSNet_gaze360.pkl',
    arch='ResNet50',
    device = select_device(device, batch_size=1)
)
 
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened correctly
if not cap.isOpened():
    raise IOError(f"Cannot open video file: {video_path}")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

if not out.isOpened():
    raise IOError(f"Cannot create video writer for: {output_path}")

frame_count = 0

with torch.no_grad():
    while True:

        # Get frame
        success, frame = cap.read()    
        start_fps = time.time()  

        if not success:
            print("Video processing completed")
            break

        frame_count += 1
        print(f"Processing frame {frame_count}/{total_frames}")

        # Process frame
        results = gaze_pipeline.step(frame)

        # Visualize output
        frame = render(frame, results)
       
        myFPS = 1.0 / (time.time() - start_fps)
        cv2.putText(frame, 'FPS: {:.1f}'.format(myFPS), (10, 20),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 0), 1, cv2.LINE_AA)

        # Write frame to output video
        out.write(frame)

cap.release()
out.release()
print(f"Output video saved to: {output_path}")

Processing frame 1/29
Processing frame 2/29
Processing frame 3/29
Processing frame 4/29
Processing frame 5/29
Processing frame 6/29
Processing frame 7/29
Processing frame 8/29
Processing frame 9/29
Processing frame 10/29
Processing frame 11/29
Processing frame 12/29
Processing frame 13/29
Processing frame 14/29
Processing frame 15/29
Processing frame 16/29
Processing frame 17/29
Processing frame 18/29
Processing frame 19/29
Processing frame 20/29
Processing frame 21/29
Processing frame 22/29
Processing frame 23/29
Processing frame 24/29
Processing frame 25/29
Processing frame 26/29
Processing frame 27/29
Processing frame 28/29
Processing frame 29/29
Video processing completed
Output video saved to: ../output_gaze/video12.mp4
